# finding reading all das files

In [1]:
%load_ext autotime
base_directory='/home/osman/aramko/'
das_directory=base_directory+'inputs/das'

time: 359 µs


In [2]:
def h5_reader(f):
    return h5py.File(f, 'r')

def time_extracter(h5file):
    return int(h5file['t'][0])

def das_extracter(f):
    return pd.DataFrame(np.array(h5_reader(f).get('das')))

time: 450 µs


In [3]:
import glob
import pandas as pd
import h5py
import swifter
import numpy as np
import sys
from scipy import signal
files=glob.glob(das_directory+"/*.h5")
filesdf=pd.DataFrame(files,columns=['file'])
chanels=list(h5_reader(filesdf['file'][0])['channel'])
filesdf['name']=filesdf['file'].swifter.apply(lambda x:x.split('/')[-1].replace('.h5',''))
#filesdf['das']=filesdf['file'].swifter.apply(lambda x:h5_reader(x).get('das'))
chanels=list(h5_reader(filesdf['file'][0])['channel'])
len(files)

918

time: 1.49 s


# finding missed data in das

In [4]:
def filename_extracter(last_file_name):
        if last_file_name[-2:]=='17':
            return str(int(last_file_name)+30)
        else :
            return str(int(last_file_name)+70)

time: 392 µs


In [5]:
filesdf['start']=filesdf['file'].swifter.apply(lambda x:time_extracter(h5_reader(x)))
filesdf=filesdf.sort_values('start').reset_index(drop=True)
all_files=pd.DataFrame(list(range(filesdf.start.min(),filesdf.start.max(),30)),columns=['start'])
all_files=all_files.merge(filesdf,how='left',on='start')
all_files['last_file_name']=all_files['name'].shift(periods=1)
all_files=all_files.fillna('-1')
all_files['output']=0
for i in all_files[all_files.name=='-1'].index:
    all_files.loc[i, 'name'] = filename_extracter(all_files.loc[i, 'last_file_name'])
    all_files.loc[i+1, 'last_file_name']=all_files.loc[i, 'name']
    all_files.loc[i, 'output']=1
all_files[all_files.output==1][['start','name']]

,start,name
483,1458306107,160318130147
484,1458306137,160318130177
485,1458306167,160318130207
486,1458306197,160318130237
487,1458306227,160318130267
...,...,...
1081,1458324047,160318174607
1082,1458324077,160318174637
1083,1458324107,160318174667
1084,1458324137,160318174697


time: 1.36 s


# splitting das files by chanels

In [6]:
numsimp=int(30000/100)
def signal_preprocessing(sig):
    lowpass_filter = signal.butter(1,Wn=0.001,btype='low',output='sos')
    a, b = signal.welch(signal.resample((signal.sosfilt(lowpass_filter,sig)),int(numsimp)), fs=0.1, scaling='spectrum')
    return b

def preprocessing_generale(file):    
    data=[]
    tmp=das_extracter(file)
    for i in tmp.columns:
        data.append(signal_preprocessing(tmp[i]))
    return pd.DataFrame([data])

time: 536 µs


In [7]:
import multiprocessing
import numpy as np
import pandas as pd
num_cores = multiprocessing.cpu_count()
with multiprocessing.Pool(num_cores) as pool:
    data_by_chanel = pd.concat(pool.map(preprocessing_generale, filesdf['file']), ignore_index=True)

time: 3min 37s


# starting processing data by channel

In [8]:
from sklearn.preprocessing import MinMaxScaler
from cuml.decomposition import PCA as cuPCA
from cuml import DBSCAN as cumlDBSCAN
scaler= MinMaxScaler()

pca_cuml = cuPCA(n_components=3,random_state=42)

db_gpu = cumlDBSCAN(eps=0.08,min_samples=2)

def signal_transform(filesdf,chanel):
    result_cuml = pca_cuml.fit_transform(scaler.fit_transform(pd.DataFrame(filesdf[chanel].values.tolist())))
    return db_gpu.fit_predict(result_cuml)

time: 4.42 s


In [9]:
for i in list(range(len(chanels))):
    filesdf['chanel'+str(chanels[i])]=signal_transform(data_by_chanel,i)

time: 18.5 s


# starting processing data for TRAINING

In [10]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
def rollback(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
                names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

time: 1.87 s


In [11]:
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion) 
    X_train = matrix[ratio:,:]
    X_test =  matrix[:ratio,:]
    Y_train = target[ratio:,:]
    Y_test =  target[:ratio,:]
    return X_train, X_test, Y_train, Y_test

time: 433 µs


# starting TRAINING

In [12]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.optimizers import  Adam
from keras.callbacks import TensorBoard
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)
rm=Adam(
    learning_rate=0.003,
)
def trainer(chanel,window,classes,x_train, x_test, y_train, y_test):
    print(chanel)
    model = Sequential()
    model.add(LSTM(50, activation='tanh',input_shape=(window,classes),return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(50,return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(classes,activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer=rm, metrics=['categorical_accuracy'])
    model.fit(x_train, y_train, epochs=10,verbose=0,validation_split=0.2)
    print('model score : ',model.evaluate(x_test, y_test,verbose=0)[1])
    model.save(base_directory+'outputs/das/models/'+chanel+'.h5')
    del model
    return (base_directory+'outputs/das/models/'+chanel+'.h5')

time: 1.18 s


In [13]:
def preprocessing_training(i):
    window=20
    chnl='chanel'+str(chanels[i])
    dataset=np_utils.to_categorical(rollback(filesdf[[chnl]].astype('str'),n_in=window))
    classes=dataset.shape[2]
    x_train, x_test, y_train, y_test = shuffle(dataset[:,0:window,:], dataset[:,-1,:], 3)
    return pd.DataFrame([[chnl,x_train, x_test, y_train, y_test,window,classes]],
                       columns=['chanel','x_train', 'x_test', 'y_train', 'y_test','window','classes'])

time: 471 µs


In [14]:
with multiprocessing.Pool(num_cores) as pool:
    data_preprocessed = pd.concat(pool.map(preprocessing_training, range(len(chanels))), ignore_index=True)

time: 1.68 s


# save models data and dataframes to use for further stapes!

In [15]:
models_by_chanel=data_preprocessed['chanel']
models_by_chanel['model']=data_preprocessed.apply(lambda x :trainer(x['chanel'],x['window'],x['classes'],
                                x['x_train'], x['x_test'], 
                                x['y_train'], x['y_test']),axis=1)

chanel1
model score :  0.9698996543884277
chanel2
model score :  0.9431437849998474
chanel3
model score :  0.9397993087768555
chanel4
model score :  0.9431437849998474
chanel5
model score :  0.9030100107192993
chanel6
model score :  0.9130434989929199
chanel7
model score :  0.8795986771583557
chanel8
model score :  0.9063544869422913
chanel9
model score :  0.8896321058273315
chanel10
model score :  0.9331103563308716
chanel11
model score :  0.9464883208274841
chanel12
model score :  0.9464883208274841
chanel13
model score :  0.9431437849998474
chanel14
model score :  0.9598662257194519
chanel15
model score :  0.9866220951080322
chanel16
model score :  0.9832776188850403
chanel17
model score :  0.9799330830574036
chanel18
model score :  0.9866220951080322
chanel19
model score :  0.9765886068344116
chanel20
model score :  0.9765886068344116
chanel21
model score :  0.9498327970504761
chanel22
model score :  0.9364548325538635
chanel23
model score :  0.9364548325538635
chanel24
model score

<ipython-input-15-8f69b441c858>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  models_by_chanel['model']=data_preprocessed.apply(lambda x :trainer(x['chanel'],x['window'],x['classes'],


In [19]:
data_preprocessed.to_pickle(base_directory+'/outputs/das/data_preprocessed.pkl')
models_by_chanel.to_pickle(base_directory+'/outputs/das/models_by_chanel.pkl')
filesdf.to_pickle(base_directory+'/outputs/das/filesdf.pkl')
data_by_chanel.to_pickle(base_directory+'/outputs/das/data_by_chanel.pkl')
filesdf.to_pickle(base_directory+'/outputs/das/filesdf.pkl')
all_files.to_pickle(base_directory+'/outputs/das/all_files.pkl')

time: 4.81 s
